In [57]:
import time
import email
import smtplib
import imaplib
def read_email_from_gmail():
    try:
        mail = imaplib.IMAP4_SSL('imap.gmail.com')
        mail.login("garglakshay631@gmail.com","abcd@1234")
        mail.select('inbox')

        typ, data = mail.search(None, 'ALL')
        print(typ)
        print(data)
        mail_ids = data[0]

        id_list = mail_ids.split()  
        print(id_list)
        first_email_id = int(id_list[0])
        print(first_email_id)
        latest_email_id = int(id_list[-1])
        print(latest_email_id)


        for i in range(latest_email_id,first_email_id, -1):
            typ, data = mail.fetch(str(i), '(RFC822)' )

            for response_part in data:
                print(response_part[-1])
                if isinstance(response_part, tuple):
                    msg = email.message_from_string(response_part[1])
                    email_subject = msg['subject']
                    email_from = msg['from']
                    print('From : ' + email_from + '\n')
                    print ('Subject : ' + email_subject + '\n')

    except (Exception, e):
        print (str(e))

In [58]:
read_email_from_gmail()

OK
[b'1 2 3 4 5 6']
[b'1', b'2', b'3', b'4', b'5', b'6']
1
6
b'Delivered-To: garglakshay631@gmail.com\r\nReceived: by 2002:a9d:11f3:0:0:0:0:0 with SMTP id y48-v6csp5047409oty;\r\n        Tue, 18 Sep 2018 04:35:20 -0700 (PDT)\r\nX-Received: by 2002:a67:8391:: with SMTP id f139-v6mr9173708vsd.6.1537270520151;\r\n        Tue, 18 Sep 2018 04:35:20 -0700 (PDT)\r\nARC-Seal: i=1; a=rsa-sha256; t=1537270520; cv=none;\r\n        d=google.com; s=arc-20160816;\r\n        b=vwtGavgfct71/uc8MTHYkyKneNiVtitpic5wkJrlgOw+fnYdj0pIvbQ5r8azIyKlPF\r\n         RuxcydMmkuvNXkfrJJd7Ogs/tzFWAy7tQ2Y/o87VOrbcWua7MoXI/8dNB1hCFX1iygas\r\n         PLwY0OxEvsCWgF6auWxLxvCNLaoDQkfNDq4Cs75wP2EKoQqC1RAjzFE1i12lRJS0BVVX\r\n         wykX5x7ju9f973QOoTzKoa6yvvOBdZhjBzqu+IBIHIb+YJ+jw20Q2RflM0toSdx9r+xN\r\n         MhVbRXfsC9E3OYdcW+fRU2ypOUEHa1DI1VvNg3wFzxRw/MzG/IfRBFgmadjLId205fsb\r\n         avQw==\r\nARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;\r\n        h=to:subject:messa

NameError: name 'e' is not defined

In [ ]:
i=[1,2,3,4,5]
a=str(i[0])
print(a)
print(type(a))